In [3]:
from sympy import symbols, Eq, solve

In [4]:

def market_drop(leveraged_x, new_leveraged_x):
    amount = 10000.
    leveraged_amount = amount * leveraged_x

    x = symbols('x')
    equation = Eq((leveraged_amount - x) / (amount - x), new_leveraged_x)
    total_amount_drop = solve(equation, x)[0]
    return total_amount_drop / amount


def upro_mes_portfolio(curr_lev):
    amount = 10000.

In [5]:
print(market_drop(3, 4))

0.333333333333333
